1. Create a JSONBin Account & Get API Key

Go to https://jsonbin.io/.

Sign up → Verify email.

Go to API Keys in the dashboard.

Copy your Master Key (this will be your "secret").

2. Create a Bin for User Data

In your dashboard, create a new bin with initial empty data:

[]


This will store an array of users.

Copy the Bin ID from the URL:
Example: https://api.jsonbin.io/v3/b/<BIN_ID>

3. Basic API Endpoints

GET → Fetch all users
GET https://api.jsonbin.io/v3/b/<BIN_ID>/latest

PUT → Update the whole bin
PUT https://api.jsonbin.io/v3/b/<BIN_ID>

Headers required:

Content-Type: application/json
X-Master-Key: YOUR_JSONBIN_KEY

4. Example in Python (Flask)

Let’s make a super simple user management API using JSONBin:

In [ ]:
import requests
from flask import Flask, request, jsonify

app = Flask(__name__)

JSONBIN_API_KEY = "your_master_key_here"
BIN_ID = "your_bin_id_here"
BASE_URL = f"https://api.jsonbin.io/v3/b/{BIN_ID}"

HEADERS = {
    "Content-Type": "application/json",
    "X-Master-Key": JSONBIN_API_KEY
}

def get_users():
    res = requests.get(f"{BASE_URL}/latest", headers=HEADERS)
    return res.json()["record"]

def save_users(users):
    requests.put(BASE_URL, headers=HEADERS, json=users)

@app.route("/register", methods=["POST"])
def register():
    data = request.json
    users = get_users()

    if any(u["username"] == data["username"] for u in users):
        return jsonify({"error": "Username already exists"}), 400

    users.append({
        "username": data["username"],
        "password": data["password"],  # NOTE: store hashed in real apps
        "role": "user"
    })

    save_users(users)
    return jsonify({"message": "User registered successfully"})

@app.route("/login", methods=["POST"])
def login():
    data = request.json
    users = get_users()

    for u in users:
        if u["username"] == data["username"] and u["password"] == data["password"]:
            return jsonify({"message": "Login successful"})
    return jsonify({"error": "Invalid credentials"}), 401

if __name__ == "__main__":
    app.run(debug=True)


5. How It Works

Register → Add a new user object to JSONBin.

Login → Fetch JSONBin, search for matching user credentials.

JSONBin acts like your database — no need to host MySQL, MongoDB, etc.

6. Limitations

Not for production — No query engine, full bin overwrite each time.

Rate limits — JSONBin’s free tier has limits (50k requests/month).

Security — Keep your X-Master-Key on the server, never in frontend JS.